In [1]:
import numpy as np
import pandas as pd

In [2]:
#!pip install openpyxl

In [3]:
#pip install pandas --upgrade

In [2]:
# Function of depression_TRD markov train calculation. 
# Inputs: inc: initial incidence; p_m: probabitly matirx (study period * 16)
def Marcov_depres_TRD(inc, p_m, period): 
    results = []
    results.append([inc,0,0,0,0,0]) 
    for t in range(period):
        p_m_t = p_m[t,:]
        t_m = [
                # NTRD      NTRD-c     TRD        TRD-c      DEAD       LISU
                [ p_m_t[4], p_m_t[1],  p_m_t[0],  0,         p_m_t[2],  p_m_t[3] ], #NTRD
                [ 0,        p_m_t[12], p_m_t[9],  0,         p_m_t[10], p_m_t[11]], #NTRD-c
                [ 0,        0,         p_m_t[8],  p_m_t[5],  p_m_t[6],  p_m_t[7] ], #TRD
                [ 0,        0,         0,         p_m_t[15], p_m_t[13], p_m_t[14]], #TRD-c
                [ 0,        0,         0,         0,         1,         0        ], #DEAD
                [ 0,        0,         0,         0,         0,         1        ]  #LISU
            ]
        results.append(np.dot(results[t], t_m)) 

    return results

For baseline + COVID-19 

In [5]:
# Define the file path, sheet name, and matrix location
file_path = ".\SCAN Burden project model for depression.xlsx"
sheet_name = "TP_BC"
data_p = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
sheet_name = "Model input"
data_i = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
sheet_name = "Cost_DPSA"
data_c = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')


Subgroup weights: 
w10_24_F_No_history:0; w25_40_F_No_history:1; w40_65_F_No_history:2; w65_F_No_history:3; w10_24_F_History:4; w25_40_F_History:5; w40_65_F_History:6; w65_F_History:7; w10_24_M_No_history:8; w25_40_M_No_history:9;w40_65_M_No_history:10; w65_M_No_history:11; w10_24_M_History:12; w25_40_M_History:13; w40_65_M_History:14; w65_M_History:15;

In [24]:
weights = data_i.iloc[51:67, 3].to_numpy()

Incidence list

In [25]:
inc = data_i.iloc[27:46, 3].to_numpy()

Period: length of the projection period: 10

In [8]:
period = 10

All-cause cost list

In [9]:
cost = data_c.iloc[0:64, 7].to_numpy()
c_m_pre = cost.reshape(4,16)
c_m = []
c_m = c_m_pre[0,:]
c_m = np.vstack((c_m, c_m_pre[2,:]))
c_m = np.vstack((c_m, c_m_pre[1,:]))
c_m = np.vstack((c_m, c_m_pre[3,:]))
zero_rows = np.zeros((2, c_m.shape[1]))
c_m = np.vstack((c_m, zero_rows))
#c_m = c_m.transpose() # the cost on the stages in different weight index: weight index 16 * stages number 6

3D matrix for all probability matrices, listing according to weight index

In [16]:
p_m_all = []
p_m_all.append(data_p.iloc[8:18, 1:17].to_numpy()) # w10_24_F_No_history: 0
p_m_all.append(data_p.iloc[26:36, 1:17].to_numpy()) # w25_40_F_No_history: 1
p_m_all.append(data_p.iloc[44:54, 1:17].to_numpy()) # w40_65_F_No_history: 2
p_m_all.append(data_p.iloc[62:72, 1:17].to_numpy()) # w65_F_No_history: 3
p_m_all.append(data_p.iloc[80:90, 1:17].to_numpy()) # w10_24_F_History: 4
p_m_all.append(data_p.iloc[98:108, 1:17].to_numpy()) # w25_40_F_History: 5
p_m_all.append(data_p.iloc[116:126, 1:17].to_numpy()) # w40_65_F_History: 6
p_m_all.append(data_p.iloc[134:144, 1:17].to_numpy()) # w65_F_History: 7
p_m_all.append(data_p.iloc[152:162, 1:17].to_numpy()) # w10_24_M_No_history: 8
p_m_all.append(data_p.iloc[170:180, 1:17].to_numpy()) # w25_40_M_No_history: 9
p_m_all.append(data_p.iloc[188:198, 1:17].to_numpy()) # w40_65_M_No_history: 10
p_m_all.append(data_p.iloc[206:216, 1:17].to_numpy()) # w65_M_No_history: 11
p_m_all.append(data_p.iloc[224:234, 1:17].to_numpy()) # w10_24_M_History: 12
p_m_all.append(data_p.iloc[242:252, 1:17].to_numpy()) # w25_40_M_History: 13
p_m_all.append(data_p.iloc[260:270, 1:17].to_numpy()) # w40_65_M_History: 14
p_m_all.append(data_p.iloc[278:288, 1:17].to_numpy()) # w65_M_History: 15

Initialize the 3D results matrix:  listing len(inc) matrices of projecstion of different year, from (2014 + current index) to (2014 + current index + period)  

In [17]:
results_2014 = []

1. For 2014 cohort (incidence index 0)

In [26]:
results_2014 = []
cost_2014 =[]

for i in range(len(weights)):
    inc_n = inc[0] * weights[i] # incidence index 0 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2014.append(result)
    cost_2014.append(np.dot(result,c_m[:,i]))

1.1 Cost of subgroup: Age: 10-24, Female, Medical history: negative. 10 year prediction from 2014

In [19]:
cost_2014[0][1:]


array([4387595.23967669, 4093170.39797464, 3838807.02743178,
       3621208.64985564, 3430063.35663501, 3258730.07094987,
       3103153.35787711, 2960748.92815189, 2829755.56455493,
       2708882.07855302])

1.2 Cost of subgroup: Age: 25-40, Female, Medical history: negative. 10 year prediction from 2014

In [20]:
cost_2014[1][1:]

array([12044058.9922097 , 11510305.95285573, 10986462.39286892,
       10507551.10803556, 10067656.81113996,  9659485.53842784,
        9277743.56911176,  8918825.47529795,  8580245.10132745,
        8260218.77385981])

1.3 Total cost of whole population. 10 year prediction from 2014

In [27]:
cost_2014_whole_po =  np.sum(cost_2014, axis=0)
cost_2014_whole_po[1:]

array([2.93276654e+08, 2.75712765e+08, 2.58179355e+08, 2.42857274e+08,
       2.29571955e+08, 2.17933314e+08, 2.07608600e+08, 1.98344735e+08,
       1.89952687e+08, 1.82290162e+08])

2. For 2015 - 2032, 10 years Closed cohort + Pandemic effect All-cause HRU cost prediction 

2015

In [28]:
results_2015 = []
cost_2015 =[]

for i in range(len(weights)):
    inc_n = inc[1] * weights[i] # incidence index 1 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2015.append(result)
    cost_2015.append(np.dot(result,c_m[:,i]))

cost_2015_whole_po =  np.sum(cost_2015, axis=0)
cost_2015_whole_po[1:]

array([2.95681949e+08, 2.77974011e+08, 2.60296801e+08, 2.44849058e+08,
       2.31454779e+08, 2.19720685e+08, 2.09311292e+08, 1.99971450e+08,
       1.91510576e+08, 1.83785207e+08])

2016

In [29]:
results_2016 = []
cost_2016 =[]

for i in range(len(weights)):
    inc_n = inc[2] * weights[i] # incidence index 2 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2016.append(result)
    cost_2016.append(np.dot(result,c_m[:,i]))

cost_2016_whole_po =  np.sum(cost_2016, axis=0)
cost_2016_whole_po[1:]

array([2.84468530e+08, 2.67432146e+08, 2.50425325e+08, 2.35563421e+08,
       2.22677106e+08, 2.11388014e+08, 2.01373387e+08, 1.92387749e+08,
       1.84247744e+08, 1.76815351e+08])

2017

In [30]:
results_2017 = []
cost_2017 =[]

for i in range(len(weights)):
    inc_n = inc[3] * weights[i] # incidence index 3 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2017.append(result)
    cost_2017.append(np.dot(result,c_m[:,i]))

cost_2017_whole_po =  np.sum(cost_2017, axis=0)
cost_2017_whole_po[1:]

array([2.92361964e+08, 2.74852854e+08, 2.57374129e+08, 2.42099836e+08,
       2.28855951e+08, 2.17253610e+08, 2.06961097e+08, 1.97726125e+08,
       1.89360251e+08, 1.81721624e+08])

2018

In [31]:
results_2018 = []
cost_2018 =[]

for i in range(len(weights)):
    inc_n = inc[4] * weights[i] # incidence index 4 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2018.append(result)
    cost_2018.append(np.dot(result,c_m[:,i]))

cost_2018_whole_po =  np.sum(cost_2018, axis=0)
cost_2018_whole_po[1:]

array([2.86704438e+08, 2.69534149e+08, 2.52393656e+08, 2.37414937e+08,
       2.24427337e+08, 2.13049513e+08, 2.02956172e+08, 1.93899907e+08,
       1.85695922e+08, 1.78205111e+08])

2019

In [32]:
results_2019 = []
cost_2019 =[]

for i in range(len(weights)):
    inc_n = inc[5] * weights[i] # incidence index 5 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2019.append(result)
    cost_2019.append(np.dot(result,c_m[:,i]))

cost_2019_whole_po =  np.sum(cost_2019, axis=0)
cost_2019_whole_po[1:]

array([3.96534970e+08, 3.72787098e+08, 3.49080438e+08, 3.28363682e+08,
       3.10400801e+08, 2.94664369e+08, 2.80704477e+08, 2.68178944e+08,
       2.56832183e+08, 2.46471797e+08])

2020

In [33]:
results_2020 = []
cost_2020 =[]

for i in range(len(weights)):
    inc_n = inc[6] * weights[i] # incidence index 6 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2020.append(result)
    cost_2020.append(np.dot(result,c_m[:,i]))

cost_2020_whole_po =  np.sum(cost_2020, axis=0)
cost_2020_whole_po[1:]

array([3.71431816e+08, 3.49187334e+08, 3.26981454e+08, 3.07576199e+08,
       2.90750481e+08, 2.76010264e+08, 2.62934121e+08, 2.51201533e+08,
       2.40573093e+08, 2.30868585e+08])

2021

In [34]:
results_2021 = []
cost_2021 =[]

for i in range(len(weights)):
    inc_n = inc[7] * weights[i] # incidence index 7 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2021.append(result)
    cost_2021.append(np.dot(result,c_m[:,i]))

cost_2021_whole_po =  np.sum(cost_2021, axis=0)
cost_2021_whole_po[1:]

array([3.72244874e+08, 3.49951699e+08, 3.27697210e+08, 3.08249478e+08,
       2.91386928e+08, 2.76614446e+08, 2.63509679e+08, 2.51751409e+08,
       2.41099703e+08, 2.31373952e+08])

2022

In [36]:
results_2022 = []
cost_2022 =[]

for i in range(len(weights)):
    inc_n = inc[8] * weights[i] # incidence index 8 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2022.append(result)
    cost_2022.append(np.dot(result,c_m[:,i]))

cost_2022_whole_po =  np.sum(cost_2022, axis=0)
cost_2022_whole_po[1:]

array([3.68755502e+08, 3.46671300e+08, 3.24625422e+08, 3.05359990e+08,
       2.88655507e+08, 2.74021500e+08, 2.61039576e+08, 2.49391526e+08,
       2.38839668e+08, 2.29205084e+08])

2023

In [37]:
results_2023 = []
cost_2023 =[]

for i in range(len(weights)):
    inc_n = inc[9] * weights[i] # incidence index 8 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2023.append(result)
    cost_2023.append(np.dot(result,c_m[:,i]))

cost_2023_whole_po =  np.sum(cost_2023, axis=0)
cost_2023_whole_po[1:]

array([3.09876580e+08, 2.91318547e+08, 2.72792718e+08, 2.56603383e+08,
       2.42566093e+08, 2.30268687e+08, 2.19359577e+08, 2.09571363e+08,
       2.00704312e+08, 1.92608076e+08])

2024

In [39]:
results_2024 = []
cost_2024 =[]

for i in range(len(weights)):
    inc_n = inc[10] * weights[i] # incidence index 8 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2024.append(result)
    cost_2024.append(np.dot(result,c_m[:,i]))

cost_2024_whole_po =  np.sum(cost_2024, axis=0)
cost_2024_whole_po[1:]

array([3.12959423e+08, 2.94216763e+08, 2.75506628e+08, 2.59156232e+08,
       2.44979290e+08, 2.32559542e+08, 2.21541902e+08, 2.11656308e+08,
       2.02701043e+08, 1.94524260e+08])

2025

In [41]:
results_2025 = []
cost_2025 =[]

for i in range(len(weights)):
    inc_n = inc[11] * weights[i] # incidence index 8 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2025.append(result)
    cost_2025.append(np.dot(result,c_m[:,i]))

cost_2025_whole_po =  np.sum(cost_2025, axis=0)
cost_2025_whole_po[1:]

array([3.15974512e+08, 2.97051283e+08, 2.78160892e+08, 2.61652974e+08,
       2.47339450e+08, 2.34800049e+08, 2.23676263e+08, 2.13695431e+08,
       2.04653889e+08, 1.96398330e+08])

2026

In [43]:
results_2026 = []
cost_2026 =[]

for i in range(len(weights)):
    inc_n = inc[12] * weights[i] # incidence index 8 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2026.append(result)
    cost_2026.append(np.dot(result,c_m[:,i]))

cost_2026_whole_po =  np.sum(cost_2026, axis=0)
cost_2026_whole_po[1:]

array([3.18921846e+08, 2.99822105e+08, 2.80755510e+08, 2.64093610e+08,
       2.49646573e+08, 2.36990207e+08, 2.25762661e+08, 2.15688730e+08,
       2.06562851e+08, 1.98230286e+08])

2027

In [44]:
results_2027 = []
cost_2027 =[]

for i in range(len(weights)):
    inc_n = inc[13] * weights[i] # incidence index 8 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2027.append(result)
    cost_2027.append(np.dot(result,c_m[:,i]))

cost_2027_whole_po =  np.sum(cost_2027, axis=0)
cost_2027_whole_po[1:]

array([3.21699793e+08, 3.02433685e+08, 2.83201011e+08, 2.66393979e+08,
       2.51821102e+08, 2.39054494e+08, 2.27729151e+08, 2.17567472e+08,
       2.08362102e+08, 1.99956957e+08])

2028

In [45]:
results_2028 = []
cost_2028 =[]

for i in range(len(weights)):
    inc_n = inc[14] * weights[i] # incidence index 8 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2028.append(result)
    cost_2028.append(np.dot(result,c_m[:,i]))

cost_2028_whole_po =  np.sum(cost_2028, axis=0)
cost_2028_whole_po[1:]

array([3.23834069e+08, 3.04440143e+08, 2.85079872e+08, 2.68161336e+08,
       2.53491777e+08, 2.40640470e+08, 2.29239991e+08, 2.19010895e+08,
       2.09744454e+08, 2.01283546e+08])

2029

In [47]:
results_2029 = []
cost_2029 =[]

for i in range(len(weights)):
    inc_n = inc[15] * weights[i] # incidence index 8 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2029.append(result)
    cost_2029.append(np.dot(result,c_m[:,i]))

cost_2029_whole_po =  np.sum(cost_2029, axis=0)
cost_2029_whole_po[1:]

array([3.26069977e+08, 3.06542146e+08, 2.87048203e+08, 2.70012853e+08,
       2.55242008e+08, 2.42301969e+08, 2.30822776e+08, 2.20523053e+08,
       2.11192632e+08, 2.02673306e+08])

2030

In [48]:
results_2030 = []
cost_2030 =[]

for i in range(len(weights)):
    inc_n = inc[16] * weights[i] # incidence index 8 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2030.append(result)
    cost_2030.append(np.dot(result,c_m[:,i]))

cost_2030_whole_po =  np.sum(cost_2030, axis=0)
cost_2030_whole_po[1:]

array([3.28475273e+08, 3.08803392e+08, 2.89165649e+08, 2.72004636e+08,
       2.57124832e+08, 2.44089340e+08, 2.32525469e+08, 2.22149769e+08,
       2.12750521e+08, 2.04168351e+08])

2031

In [50]:
results_2031 = []
cost_2031 =[]

for i in range(len(weights)):
    inc_n = inc[17] * weights[i] # incidence index 8 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2031.append(result)
    cost_2031.append(np.dot(result,c_m[:,i]))

cost_2031_whole_po =  np.sum(cost_2031, axis=0)
cost_2031_whole_po[1:]

array([3.30575671e+08, 3.10778001e+08, 2.91014687e+08, 2.73743940e+08,
       2.58768989e+08, 2.45650142e+08, 2.34012327e+08, 2.23570281e+08,
       2.14110930e+08, 2.05473883e+08])

2032

In [52]:
results_2032 = []
cost_2032 =[]

for i in range(len(weights)):
    inc_n = inc[18] * weights[i] # incidence index 8 * weight index i 
    p_m = p_m_all[i] # weight index i
    result = np.array(Marcov_depres_TRD(inc_n, p_m, period))
    results_2032.append(result)
    cost_2032.append(np.dot(result,c_m[:,i]))

cost_2032_whole_po =  np.sum(cost_2032, axis=0)
cost_2032_whole_po[1:]

array([3.32743825e+08, 3.12816307e+08, 2.92923371e+08, 2.75539350e+08,
       2.60466182e+08, 2.47261293e+08, 2.35547149e+08, 2.25036616e+08,
       2.15515224e+08, 2.06821529e+08])

3. All patient Open cohort + Pandemic effect 2023 - 2032 cost prediction

In [67]:
open_cost_2023 = cost_2014_whole_po[-1] + cost_2015_whole_po[-2] + cost_2016_whole_po[-3] + cost_2017_whole_po[-4] + cost_2018_whole_po[-5]+ cost_2019_whole_po[-6] + cost_2020_whole_po[-7]+ cost_2021_whole_po[-8] + cost_2022_whole_po[-9]  + cost_2023_whole_po[-10]
open_cost_2023

2588421186.919001